### Cell 1 - 連接 Qdrant

In [1]:
from qdrant_client import QdrantClient

client = QdrantClient(url="http://localhost:6333")
print("連接成功！")

連接成功！


### Cell 2 - 建立 Collection

In [2]:
from qdrant_client.models import Distance, VectorParams

client.create_collection(
    collection_name="test_collection",
    # size 記得跟 model 的 embedding size 一致
    vectors_config=VectorParams(size=4096, distance=Distance.COSINE),
)
print("Collection 建立成功！")

Collection 建立成功！


### Cell 3 - 定義取得 Embedding 的函式

In [3]:
import requests

def get_embedding(texts):
    data = {
        "texts": texts,
        "normalize": True,
        "batch_size": 32
    }
    response = requests.post("https://ws-04.wade0426.me/embed", json=data)
    
    print(f"狀態碼: {response.status_code}")
    
    if response.status_code == 200:
        result = response.json()
        print(f"維度: {result['dimension']}")
        return result['embeddings']
    else:
        print(f"錯誤: {response.json()}")
        return None

### Cell 4 - 插入資料

In [4]:
from qdrant_client.models import PointStruct

# 取得 embedding
text = "人工智慧很有趣"
embeddings = get_embedding([text])

# 注意 ID 舊的資料會被完全覆蓋
# 插入單筆資料
client.upsert(
    collection_name="test_collection",
    points=[
        PointStruct(
            id=1,  # 唯一識別碼
            vector=embeddings[0],  # 4096 維向量
            payload={"text": text, "metadata": "其他資訊"}  # 可選的附加資料
        )
    ]
)
print("資料插入成功！")

狀態碼: 200
維度: 4096
資料插入成功！


### Cell 5 - 搜尋相似資料

In [5]:
texts = ["AI 有什麼好處？"]
query_vector = get_embedding(texts)[0]

# 執行搜尋
search_result = client.query_points(
    collection_name="test_collection",
    query=query_vector,
    limit=3  # 只回傳最相似的前 3 筆
)

# 顯示結果
for point in search_result.points:
    print(f"ID: {point.id}")
    print(f"相似度分數 (Score): {point.score}")
    print(f"內容: {point.payload['text']}")
    print("---")

狀態碼: 200
維度: 4096
ID: 1
相似度分數 (Score): 0.70402336
內容: 人工智慧很有趣
---


### Cell 6 - 帶 Filter 的搜尋（進階）


In [6]:
from qdrant_client.models import Filter, FieldCondition, Range

results = client.query_points(
    collection_name="test_collection",
    query=query_vector,
    query_filter=Filter(
        must=[
            FieldCondition(
                key='year',
                range=Range(
                    gte=3,   # 大於等於 3
                    lte=10   # 小於等於 10
                )
            )
        ]
    ),
    limit=5
)

# 查看結果
for point in results.points:
    print(f"ID: {point.id}")
    print(f"Payload: {point.payload}")
    print("-" * 30)